In [1]:
# !pip install tf-agents --user -q

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import functools
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.environments import stationary_stochastic_per_arm_py_environment as p_a_env
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import tf_py_environment
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.specs import tensor_spec
from tf_agents.trajectories import time_step as ts
import tensorflow_datasets as tfds
from pprint import pprint

nest = tf.nest

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

### movies data

In [3]:
movies = tfds.load("movielens/100k-movies", split="train")

for x in movies.batch(1).take(1):
    pprint(x)

{'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[4]])>,
 'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'1681'], dtype=object)>,
 'movie_title': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'You So Crazy (1994)'], dtype=object)>}


### user and ratings data

In [4]:
ratings = tfds.load("movielens/100k-ratings", split="train")

for x in ratings.batch(1).take(1):
    pprint(x)

{'bucketized_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([45.], dtype=float32)>,
 'movie_genres': <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[7]])>,
 'movie_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'357'], dtype=object)>,
 'movie_title': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b"One Flew Over the Cuckoo's Nest (1975)"], dtype=object)>,
 'raw_user_age': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([46.], dtype=float32)>,
 'timestamp': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([879024327])>,
 'user_gender': <tf.Tensor: shape=(1,), dtype=bool, numpy=array([ True])>,
 'user_id': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'138'], dtype=object)>,
 'user_occupation_label': <tf.Tensor: shape=(1,), dtype=int64, numpy=array([4])>,
 'user_occupation_text': <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'doctor'], dtype=object)>,
 'user_rating': <tf.Tensor: shape=(1,), dtype=float32, numpy=array([4.], dtype=float32)>,

#### Let's make this simple and load up movielens that has features
We will only consider for this example
1) The movie genere as an Arm feature (we will concatenate multiple genres)
2) The user occupation and age bucket labels for the overall context features

We need to load the data, get the ratings - light EDA for us to get cardnality of the dataset as well as lookups for the 

In [5]:
# Get the unique movies and users
unique_movie_ids = ratings.map(lambda x: x["movie_id"])
unique_movie_ids = np.unique([x.numpy() for x in unique_movie_ids])
MOVIELENS_NUM_MOVIES = len(unique_movie_ids)


print(f"len(unique_movie_ids) : {len(unique_movie_ids)}")
print(f"unique_movie_ids      : {unique_movie_ids[:2]}")

len(unique_movie_ids) : 1682
unique_movie_ids      : [b'1' b'10']


In [6]:
unique_user_ids = ratings.map(lambda x: x["user_id"])
unique_user_ids = np.unique([x.numpy() for x in unique_user_ids])
MOVIELENS_NUM_USERS = len(unique_user_ids)


print(f"len(unique_user_ids) : {len(unique_user_ids)}")
print(f"unique_user_ids      : {unique_user_ids[:2]}")

len(unique_user_ids) : 943
unique_user_ids      : [b'1' b'10']


In [7]:
## Get the unnique set of user buckets and create a lookup table

In [8]:
from typing import Dict

def get_dictionary_lookup_by_tf_data_key(key: str) -> Dict:
    tensor = ratings.map(lambda x: x[key])
    unique_elems = set()
    for x in tensor:
        val = x.numpy()
        if type(val) is np.ndarray: # if multi dimesnional only grab first one
            val = val[0]
        unique_elems.add(val)
    
    #return a dictionary of keys by integer values for the feature space
    return {val: i for i, val in enumerate(unique_elems)}


In [9]:
user_age_lookup = get_dictionary_lookup_by_tf_data_key('bucketized_user_age')
user_age_dim = len(user_age_lookup)

In [10]:
user_age_lookup

{1.0: 0, 35.0: 1, 45.0: 2, 18.0: 3, 50.0: 4, 56.0: 5, 25.0: 6}

In [11]:
user_occ_lookup = get_dictionary_lookup_by_tf_data_key('user_occupation_text')
user_occ_dim = len(user_occ_lookup)

In [12]:
user_occ_lookup

{b'student': 0,
 b'salesman': 1,
 b'programmer': 2,
 b'educator': 3,
 b'homemaker': 4,
 b'writer': 5,
 b'executive': 6,
 b'artist': 7,
 b'healthcare': 8,
 b'technician': 9,
 b'doctor': 10,
 b'marketing': 11,
 b'scientist': 12,
 b'lawyer': 13,
 b'engineer': 14,
 b'entertainment': 15,
 b'administrator': 16,
 b'librarian': 17,
 b'other': 18,
 b'none': 19,
 b'retired': 20}

In [13]:
movie_gen_lookup = get_dictionary_lookup_by_tf_data_key('movie_genres')
movie_gen_dim = len(movie_gen_lookup)

In [14]:
movie_gen_lookup

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18}

In [15]:

 #from https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/environments/dataset_utilities.py#L153
    
# def load_movielens_data(data_file, delimiter=','):
#     """Loads the movielens data and returns the ratings matrix."""
#     ratings_matrix = np.zeros([MOVIELENS_NUM_USERS, MOVIELENS_NUM_MOVIES])
#     with tf.io.gfile.GFile(data_file, 'r') as infile:
#     # The file is a csv with rows containing:
#     # user id | item id | rating | timestamp
#     reader = csv.reader(infile, delimiter=delimiter)
#     for row in reader:
#         user_id, item_id, rating, _ = row
#         ratings_matrix[int(user_id) - 1, int(item_id) - 1] = float(rating)
#     return ratings_matrix



def load_movielens_data(ratings_dataset):
    # ratings = tfds.load("movielens/100k-ratings", split="train")
    ratings_matrix = np.zeros([MOVIELENS_NUM_USERS, MOVIELENS_NUM_MOVIES])
    local_data = ratings_dataset.map(lambda x: {'user_id': x['user_id']
                                                 ,'movie_id':  x['movie_id']
                                                 ,'user_rating':  x['user_rating']
                                                 ,'bucketized_user_age': x['bucketized_user_age']
                                                 ,'user_occupation_text': x['user_occupation_text']
                                                 ,'movie_genres': x['movie_genres'][0]
                                               }
                                                                         )
    user_age_int = []
    user_occ_int = []
    mov_gen_int = []
    for row in local_data:
        ratings_matrix[int(row['user_id'].numpy()) - 1, int(row['movie_id'].numpy()) - 1] = float(row['user_rating'].numpy())
        user_age_int.append(user_age_lookup[row['bucketized_user_age'].numpy()])
        user_occ_int.append(user_occ_lookup[row['user_occupation_text'].numpy()])
        mov_gen_int.append(movie_gen_lookup[row['movie_genres'].numpy()])
    return ratings_matrix, np.array(user_age_int), np.array(user_occ_int), np.array(mov_gen_int)
    

In [16]:
ratings_matrix, user_age_int, user_occ_int, mov_gen_int = load_movielens_data(ratings)

In [17]:
from tf_agents.bandits.specs import utils as bandit_spec_utils

## Replicate an agent using the above data

https://github.com/tensorflow/agents/blob/master/tf_agents/bandits/environments/movielens_per_arm_py_environment.py

Create an arm spec from this utility function
https://www.tensorflow.org/agents/api_docs/python/tf_agents/specs/bandit_spec_utils/create_per_arm_observation_spec

In [18]:
# Example observation spec from above
# There are 20 user occupations and 7 age buckets. This makes our global dimension 27
# There are 19 genres, and that will be the arm dimension for this example

from tf_agents.specs.bandit_spec_utils import create_per_arm_observation_spec as create_obs_spec
create_obs_spec(
    global_dim = 1,
    per_arm_dim = 2,
    max_num_actions = 10,
    add_num_actions_feature = False
) 

{'global': TensorSpec(shape=(1,), dtype=tf.float32, name=None),
 'per_arm': TensorSpec(shape=(10, 2), dtype=tf.float32, name=None)}

In [19]:
ratings.cardinality().numpy()

100000

In [66]:
"""Class implementation of the per-arm MovieLens Bandit environment."""
from __future__ import absolute_import

import random
from typing import Optional, Text
import gin
import numpy as np

from tf_agents.bandits.environments import bandit_py_environment
from tf_agents.bandits.environments import dataset_utilities
from tf_agents.bandits.specs import utils as bandit_spec_utils
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts


GLOBAL_KEY = bandit_spec_utils.GLOBAL_FEATURE_KEY
PER_ARM_KEY = bandit_spec_utils.PER_ARM_FEATURE_KEY


@gin.configurable
class MovieLensPerArmPyEnvironment(bandit_py_environment.BanditPyEnvironment):
    """Implements the per-arm version of the MovieLens Bandit environment.

    This environment implements the MovieLens 100K dataset, available at:
    https://www.kaggle.com/prajitdatta/movielens-100k-dataset

    This dataset contains 100K ratings from 943 users on 1682 items.
    This csv list of:
    user id | item id | rating | timestamp.
    This environment computes a low-rank matrix factorization (using SVD) of the
    data matrix `A`, such that: `A ~= U * Sigma * V^T`.

    The environment uses the rows of `U` as global (or user) features, and the
    rows of `V` as per-arm (or movie) features.

    The reward of recommending movie `v` to user `u` is `u * Sigma * v^T`.
    """

    def __init__(self,
               dataset = ratings,
               rank_k: int = 2,
               batch_size: int = 10,
               num_actions: int = 100,
               name: Optional[Text] = 'movielens_per_arm'):
        """Initializes the Per-arm MovieLens Bandit environment.

        Args:
          data_dir: (string) Directory where the data lies (in text form).
          rank_k : (int) Which rank to use in the matrix factorization. This will
            also be the feature dimension of both the user and the movie features.
          batch_size: (int) Number of observations generated per call.
          num_actions: (int) How many movies to choose from per round.
          csv_delimiter: (string) The delimiter to use in loading the data csv file.
          name: (string) The name of this environment instance.
        """
        self._batch_size = batch_size
        self._num_actions = num_actions
        self.rank_k = rank_k

        # Compute the matrix factorization.
        # self._data_matrix = dataset_utilities.load_movielens_data(
        #     data_dir, delimiter=csv_delimiter)

        self._data_matrix, self._user_age_int, self._user_occ_int, self._mov_gen_int = load_movielens_data(ratings)
        self._num_users, self._num_movies = self._data_matrix.shape

        # Compute the SVD.
        u, s, vh = np.linalg.svd(self._data_matrix, full_matrices=False)

        # Keep only the largest singular values.
        self._u_hat = u[:, :rank_k].astype(np.float32)
        self._s_hat = s[:rank_k].astype(np.float32)
        self._v_hat = np.transpose(vh[:rank_k]).astype(np.float32)

        self._approx_ratings_matrix = np.matmul(self._u_hat * self._s_hat,
                                                np.transpose(self._v_hat))

        self._action_spec = array_spec.BoundedArraySpec(
            shape=(),
            dtype=np.int32,
            minimum=0,
            maximum=num_actions - 1,
            name='action')
        observation_spec = {
            GLOBAL_KEY:
                array_spec.ArraySpec(shape=[rank_k+2], dtype=np.float32), #creating +space for user age and occupation
            PER_ARM_KEY:
                array_spec.ArraySpec(
                    shape=[num_actions, rank_k+1], dtype=np.float32), #creating +1 space for movie genre
        }
        self._time_step_spec = ts.time_step_spec(observation_spec)

        self._current_user_indices = np.zeros(batch_size, dtype=np.int32)
        self._previous_user_indices = np.zeros(batch_size, dtype=np.int32)

        self._current_movie_indices = np.zeros([batch_size, num_actions],
                                               dtype=np.int32)
        self._previous_movie_indices = np.zeros([batch_size, num_actions],
                                                dtype=np.int32)

        self._observation = {
            GLOBAL_KEY:
                np.zeros([batch_size, rank_k+2]), #making space like above for dimensions
            PER_ARM_KEY:
                np.zeros([batch_size, num_actions, rank_k+1]),
        }

        super(MovieLensPerArmPyEnvironment, self).__init__(
            observation_spec, self._action_spec, name=name)

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def batched(self):
        return True

    def _observe(self):
        sampled_user_indices = np.random.randint(
            self._num_users, size=self._batch_size)
        self._previous_user_indices = self._current_user_indices
        self._current_user_indices = sampled_user_indices

        sampled_movie_indices = np.array([
            random.sample(range(self._num_movies), self._num_actions)
            for _ in range(self._batch_size)
        ])
        sampled_user_ages = self._user_age_int[sampled_user_indices]
        sampled_user_occ = self._user_occ_int[sampled_user_indices]
        combined_user_features = np.concatenate((self._u_hat[sampled_user_indices]
                                                 , sampled_user_ages.reshape(-1,1)
                                                 , sampled_user_occ.reshape(-1,1)), axis=1)
        # current_users = combined_user_features.reshape([self._batch_size, self.rank_k+2])
        
        movie_index_vector = sampled_movie_indices.reshape(-1)
        print(movie_index_vector.shape)
        flat_genre_list = self._mov_gen_int[movie_index_vector] #shape of 1
        flat_movie_list = self._v_hat[movie_index_vector] #shape of 2
        combined_movie_features = np.concatenate((flat_movie_list,flat_genre_list.reshape(-1,1)), axis=1)
        current_movies = combined_movie_features.reshape(
            [self._batch_size, self._num_actions, self.rank_k+1])

        self._previous_movie_indices = self._current_movie_indices
        self._current_movie_indices = sampled_movie_indices

        batched_observations = {
            GLOBAL_KEY:
                combined_user_features,
            PER_ARM_KEY:
                current_movies,
        }
        return batched_observations

    def _apply_action(self, action):
        chosen_arm_indices = self._current_movie_indices[range(self._batch_size),
                                                         action]
        return self._approx_ratings_matrix[self._current_user_indices,
                                           chosen_arm_indices]

    def _rewards_for_all_actions(self):
        rewards_matrix = self._approx_ratings_matrix[
            np.expand_dims(self._previous_user_indices, axis=-1),
            self._previous_movie_indices]
        return rewards_matrix

    def compute_optimal_action(self):
        return np.argmax(self._rewards_for_all_actions(), axis=-1)

    def compute_optimal_reward(self):
        return np.max(self._rewards_for_all_actions(), axis=-1)

In [67]:
env = MovieLensPerArmPyEnvironment()

In [68]:
env.reset().observation['global']

(1000,)


array([[-4.68955468e-03,  1.15368254e-02,  0.00000000e+00,
         0.00000000e+00],
       [-1.03842514e-02, -9.09375027e-03,  4.00000000e+00,
         3.00000000e+00],
       [-5.50003052e-02, -2.59428471e-02,  3.00000000e+00,
         0.00000000e+00],
       [-4.29374538e-03, -2.62549985e-02,  2.00000000e+00,
         3.00000000e+00],
       [-2.11818181e-02, -3.80838960e-02,  4.00000000e+00,
         1.40000000e+01],
       [-3.31762508e-02,  2.35662591e-02,  6.00000000e+00,
         1.80000000e+01],
       [-5.02946274e-03, -2.92349514e-02,  3.00000000e+00,
         0.00000000e+00],
       [-3.83273163e-03, -1.97077598e-02,  3.00000000e+00,
         1.80000000e+01],
       [-4.78417985e-03, -1.07728569e-02,  3.00000000e+00,
         1.40000000e+01],
       [-6.78159371e-02,  5.32993756e-04,  6.00000000e+00,
         0.00000000e+00]])

In [69]:
print('observation spec: ', env.observation_spec())
print('\nAn observation: ', env.reset().observation)

observation spec:  {'global': ArraySpec(shape=(4,), dtype=dtype('float32'), name=None), 'per_arm': ArraySpec(shape=(100, 3), dtype=dtype('float32'), name=None)}
(1000,)

An observation:  {'global': array([[-6.27098745e-03, -1.67641863e-02,  3.00000000e+00,
         0.00000000e+00],
       [-2.97608413e-03, -1.70790255e-02,  1.00000000e+00,
         1.70000000e+01],
       [-1.42527726e-02, -1.05242254e-02,  1.00000000e+00,
         1.10000000e+01],
       [-3.44458502e-03, -1.78005211e-02,  1.00000000e+00,
         1.80000000e+01],
       [-1.50922006e-02, -6.70611635e-02,  6.00000000e+00,
         1.80000000e+01],
       [-1.11539718e-02,  5.84645150e-03,  6.00000000e+00,
         1.60000000e+01],
       [-2.79517733e-02, -2.42402889e-02,  2.00000000e+00,
         1.60000000e+01],
       [-1.35534508e-02, -4.30957861e-02,  3.00000000e+00,
         2.00000000e+00],
       [-2.55223755e-02, -1.27906390e-02,  3.00000000e+00,
         1.50000000e+01],
       [-3.76718380e-02,  8.15294962e

### Now that the environment is created, let's optimize

Taken from here
https://github.com/tensorflow/agents/blob/5e5915b0a3650a15e82e77af6e37f41a6c744689/tf_agents/bandits/agents/examples/v2/train_eval_movielens.py#L84

In [70]:
import functools
import os
from absl import app
from absl import flags

import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import
from tf_agents.bandits.agents import dropout_thompson_sampling_agent as dropout_ts_agent
from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.agents import linear_thompson_sampling_agent as lin_ts_agent
from tf_agents.bandits.agents import neural_epsilon_greedy_agent as eps_greedy_agent
from tf_agents.bandits.agents.examples.v2 import trainer
from tf_agents.bandits.environments import environment_utilities
from tf_agents.bandits.environments import movielens_per_arm_py_environment
from tf_agents.bandits.environments import movielens_py_environment
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.bandits.networks import global_and_arm_feature_network
from tf_agents.environments import tf_py_environment
from tf_agents.networks import q_network

BATCH_SIZE = 8
TRAINING_LOOPS = 20000
STEPS_PER_LOOP = 2

RANK_K = 20
NUM_ACTIONS = 20

# LinUCB agent constants.

AGENT_ALPHA = 10.0

# epsilon Greedy constants.

EPSILON = 0.05
LAYERS = (50, 50, 50)
LR = 0.005

# Dropout TS constants.
DROPOUT_RATE = 0.2

In [71]:
tf.compat.v1.enable_v2_behavior()

In [72]:
env = MovieLensPerArmPyEnvironment(
        rank_k=RANK_K,
        batch_size=BATCH_SIZE,
        num_actions=NUM_ACTIONS,
)
environment = tf_py_environment.TFPyEnvironment(env)

### Note we will be using the reward function with this utility function

```python
@gin.configurable
def compute_optimal_reward_with_movielens_environment(observation, environment):
  """Helper function for gin configurable Regret metric."""
  del observation
  return tf.py_function(environment.compute_optimal_reward, [], tf.float32)

@gin.configurable
def compute_optimal_action_with_movielens_environment(observation,
                                                      environment,
                                                      action_dtype=tf.int32):
  """Helper function for gin configurable SuboptimalArms metric."""
  del observation
  return tf.py_function(environment.compute_optimal_action, [], action_dtype)
```

In [73]:
optimal_reward_fn = functools.partial(
      environment_utilities.compute_optimal_reward_with_movielens_environment,
      environment=environment)

optimal_action_fn = functools.partial(
  environment_utilities.compute_optimal_action_with_movielens_environment,
  environment=environment)

### Below we will try different agents by selecting one of the enumerated types:

```python
flags.DEFINE_enum(
    'agent', 'LinUCB', ['LinUCB', 'LinTS', 'epsGreedy', 'DropoutTS'],
    'Which agent to use. Possible values: `LinUCB`, `LinTS`, `epsGreedy`,'
    ' `DropoutTS`.')
```

In [74]:
AGENT_TYPE = 'LinUCB'

In [75]:
if AGENT_TYPE == 'LinUCB':
    agent = lin_ucb_agent.LinearUCBAgent(
        time_step_spec=environment.time_step_spec(),
        action_spec=environment.action_spec(),
        tikhonov_weight=0.001,
        alpha=AGENT_ALPHA,
        dtype=tf.float32,
        accepts_per_arm_features=True)

elif AGENT_TYPE == 'LinTS':
    agent = lin_ts_agent.LinearThompsonSamplingAgent(
        time_step_spec=environment.time_step_spec(),
        action_spec=environment.action_spec(),
        dtype=tf.float32,
        accepts_per_arm_features=True)

elif AGENT_TYPE == 'epsGreedy':
    network = (
      global_and_arm_feature_network
      .create_feed_forward_dot_product_network(
          environment.time_step_spec().observation,
          global_layers=LAYERS,
          arm_layers=LAYERS))

    agent = eps_greedy_agent.NeuralEpsilonGreedyAgent(
        time_step_spec=environment.time_step_spec(),
        action_spec=environment.action_spec(),
        reward_network=network,
        optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR),
        epsilon=EPSILON,
        emit_policy_info='predicted_rewards_mean',
        info_fields_to_inherit_from_greedy=['predicted_rewards_mean'])

elif AGENT_TYPE == 'DropoutTS':
    train_step_counter = tf.compat.v1.train.get_or_create_global_step()

    def dropout_fn():
        return tf.math.maximum(
          tf.math.reciprocal_no_nan(1.01 +
                                    tf.cast(train_step_counter, tf.float32)),
          0.0003)

    agent = dropout_ts_agent.DropoutThompsonSamplingAgent(
        time_step_spec=environment.time_step_spec(),
        action_spec=environment.action_spec(),
        dropout_rate=dropout_fn,
        network_layers=LAYERS,
        optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=LR))

regret_metric = tf_bandit_metrics.RegretMetric(optimal_reward_fn)
suboptimal_arms_metric = tf_bandit_metrics.SuboptimalArmsMetric(
  optimal_action_fn)

### Now train the MAB Agent

Create a local checkpoint folder if you already have not
!mkdir checkpoint

In [76]:
# !mkdir checkpoint

In [77]:
trainer.train(
      root_dir='checkpoint',
      agent=agent,
      environment=environment,
      training_loops=TRAINING_LOOPS,
      steps_per_loop=STEPS_PER_LOOP,
      additional_metrics=[regret_metric, suboptimal_arms_metric])

(160,)
(160,)


ValueError: Tensor conversion requested dtype float32 for Tensor with dtype float64: <tf.Tensor: shape=(8, 22), dtype=float64, numpy=
array([[-2.85286382e-02, -8.30348656e-02,  3.75065953e-03,
         2.72332150e-02,  5.02408035e-02, -4.55792844e-02,
        -5.30029275e-02,  6.67323694e-02,  4.98623326e-02,
        -6.48648757e-03, -4.36089113e-02, -2.25173049e-02,
         1.53402111e-03,  4.71154377e-02, -1.16039803e-02,
         3.48937027e-02, -3.36154997e-02,  5.44842742e-02,
        -5.66345733e-03, -3.53569910e-02,  1.00000000e+00,
         1.60000000e+01],
       [-1.32478410e-02,  4.16449225e-03,  2.51094010e-02,
         2.79174931e-02,  3.89474770e-03, -1.92512367e-02,
        -1.16578955e-02,  3.73599993e-04, -2.27175280e-02,
        -2.44750548e-03,  2.26559527e-02, -9.07483138e-03,
         2.50538439e-02,  8.02389439e-03,  8.56140105e-04,
         1.23351971e-02,  1.48605853e-02,  5.15011661e-02,
         9.64668696e-04,  1.86574403e-02,  6.00000000e+00,
         3.000